In [ ]:
from waymo_open_dataset import dataset_pb2

import json
import os

import numpy as np
import tensorflow as tf
from PIL import Image
from tqdm import tqdm
from waymo_open_dataset import label_pb2
from waymo_open_dataset.protos import camera_segmentation_pb2 as cs_pb2
from waymo_open_dataset.utils import box_utils


import matplotlib.pyplot as plt

In [ ]:
from pathlib import Path


WOPD_DATA_ROOT = Path("/media/nvme1/waymo_perception/training")


tfrecords_file_list = list(WOPD_DATA_ROOT.glob("*.tfrecord"))

In [ ]:
import io
from pyquaternion import Quaternion

from d123.common.geometry.base import StateSE3
from d123.common.geometry.bounding_box.bounding_box import BoundingBoxSE3

from waymo_open_dataset.utils import frame_utils


# Frame attributes:
#   context: <class 'waymo_open_dataset.dataset_pb2.Context'>
#   timestamp_micros: <class 'int'>
#   pose: <class 'waymo_open_dataset.dataset_pb2.Transform'>
#   images: List with 5 images
#   lasers: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 5
#   laser_labels: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 0
#   projected_lidar_labels: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 0
#   camera_labels: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 0
#   no_label_zones: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 0
#   map_features: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 0
#   map_pose_offset: <class 'waymo_open_dataset.protos.vector_pb2.Vector3d'>

file_idx = 1
pathname = tfrecords_file_list[file_idx]
dataset = tf.data.TFRecordDataset(pathname, compression_type="")
num_frames = sum(1 for _ in dataset)


def read_jpg_image(data: bytes) -> np.ndarray:
    """Read a JPEG image from bytes and return it as a numpy array."""
    image = Image.open(io.BytesIO(data))
    return np.array(image)


ego_state_se3s = []
front_images = []
dataset = tf.data.TFRecordDataset(pathname, compression_type="")

boxes = []

for frame_idx, data in enumerate(dataset):

    frame = dataset_pb2.Frame()
    frame.ParseFromString(data.numpy())
    print("Frame attributes:")
    for field in frame.DESCRIPTOR.fields:
        field_name = field.name
        if hasattr(frame, field_name):
            value = getattr(frame, field_name)
            if field_name != "images":  # Don't print the whole image data
                print(f"  {field_name}: {type(value)}")
                if hasattr(value, "__len__") and not isinstance(value, (str, bytes)):
                    print(f"    Length: {len(value)}")
            else:
                print(f"  {field_name}: List with {len(value)} images")

    # # plt.show()
    if frame_idx == 0:
        break

ego_state_se3s = np.array(ego_state_se3s, dtype=np.float64)

In [ ]:
frame.context.stats.location

In [ ]:
dataset = tf.data.TFRecordDataset(pathname, compression_type="")
for frame_idx, data in enumerate(dataset):
    frame = dataset_pb2.Frame()
    frame.ParseFromString(data.numpy())
    break

driveways = []
stop_signs = []

road_edges = []
road_lines = []
lanes = []
for map_feature in frame.map_features:

    if map_feature.HasField("lane"):
        lanes.append(
            np.array([[p.x, p.y, p.z] for p in map_feature.lane.polyline])
        )
    elif map_feature.HasField("road_line"):
        road_lines.append(
            np.array([[p.x, p.y, p.z] for p in map_feature.road_line.polyline])
        )
    elif map_feature.HasField("road_edge"):
        road_edges.append(
            np.array([[p.x, p.y, p.z] for p in map_feature.road_edge.polyline])
        )
    elif map_feature.HasField("stop_sign"):
        # print(map_feature)
        # outline = np.array(
        #     [[p.x, p.y, p.z] for p in map_feature.stop_sign.polygon]
        # )
        # stop_signs.append(outline)
        pass
    elif map_feature.HasField("crosswalk"):
        outline = np.array(
            [[p.x, p.y, p.z] for p in map_feature.crosswalk.polygon]
        )
    elif map_feature.HasField("speed_bump"):
        pass
    elif map_feature.HasField("driveway"):
        # print(map_feature.driveway)
        outline = np.array(
            [[p.x, p.y, p.z] for p in map_feature.driveway.polygon]
        )
        driveways.append(outline)

    # print(f"Roadline: {map_feature.road_line}")
    # break

# print(frame.map_pose_offset)


In [ ]:


from d123.common.visualization.matplotlib.utils import add_non_repeating_legend_to_ax


fig, ax = plt.subplots(figsize=(10, 10))

for road_edge in road_edges:
    # print(len(driveway))
    ax.plot(road_edge[:, 0], road_edge[:, 1], color="blue", label="road_edge")

for lane in lanes:
    # print(len(driveway))
    ax.plot(lane[:, 0], lane[:, 1], color="gray", label="lane")


for road_line in road_lines:
    # print(len(driveway))
    ax.plot(road_line[:, 0], road_line[:, 1], color="orange", label="road_line")

for driveway in driveways:
    # print(len(driveway))
    ax.plot(driveway[:, 0], driveway[:, 1], color="green", label="driveway")


for crosswalk in crosswalks:
    # print(len(driveway))
    ax.plot(crosswalk[:, 0], crosswalk[:, 1], color="green", label="driveway")


add_non_repeating_legend_to_ax(ax)

In [ ]:
from d123.common.geometry.line.polylines import Polyline3D
import numpy as np


polyline_3d = Polyline3D.from_array(lanes[0])


# Create left and right boundaries for the lane

def create_lane_boundaries(polyline_3d, width=2.0):
    """
    Create left and right boundaries for a lane by offsetting perpendicular to the heading.
    
    Args:
        polyline_3d: Polyline3D object representing the centerline
        width: Total width of the lane in meters
    
    Returns:
        left_boundary: numpy array representing the left boundary
        right_boundary: numpy array representing the right boundary
    """
    points = polyline_3d.array
    half_width = width / 2.0
    
    # Calculate the direction vectors between consecutive points
    directions = np.diff(points, axis=0)
    
    # Normalize the direction vectors
    directions_norm = np.linalg.norm(directions, axis=1, keepdims=True)
    directions_normalized = directions / directions_norm
    
    # Calculate perpendicular vectors in the xy plane (z remains 0)
    perpendiculars = np.zeros_like(directions)
    perpendiculars[:, 0] = -directions_normalized[:, 1]  # -dy
    perpendiculars[:, 1] = directions_normalized[:, 0]   # dx
    
    # Create boundaries (need to handle the last point separately)
    left_boundary = points[:-1] + perpendiculars * half_width
    right_boundary = points[:-1] - perpendiculars * half_width
    
    # Handle the last point based on the last direction
    last_perp = perpendiculars[-1]
    left_boundary = np.vstack([left_boundary, points[-1] + last_perp * half_width])
    right_boundary = np.vstack([right_boundary, points[-1] - last_perp * half_width])
    
    return left_boundary, right_boundary

# Create the boundaries with a 4m wide lane
left_boundary, right_boundary = create_lane_boundaries(polyline_3d, width=4.0)

# Plot the centerline and the boundaries
plt.figure(figsize=(12, 8))
plt.plot(polyline_3d.array[:, 0], polyline_3d.array[:, 1], 'g-', label='Centerline')
plt.plot(left_boundary[:, 0], left_boundary[:, 1], 'b-', label='Left Boundary')
plt.plot(right_boundary[:, 0], right_boundary[:, 1], 'r-', label='Right Boundary')
plt.axis('equal')
plt.grid(True)
plt.legend()
plt.title('Lane with Left and Right Boundaries')



In [ ]:
for driveway in driveways:
    # print(len(driveway))
    distance = abs(driveway[:, -1].min() - driveway[:, -1].max())
    if distance > 1:
        print(driveway[:, -1])
        plt.plot(driveway[:, 0], driveway[:, 1], "r-", color="red")
    else:
        plt.plot(driveway[:, 0], driveway[:, 1], "r-", color="blue")


In [ ]:
import matplotlib.pyplot as plt

for polyline in polylines:
    try:
        plt.plot(polyline[:, 0], polyline[:, 1])
    except Exception as e:
        print(f"Error plotting polyline: {polyline.shape}")
